In [14]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-23 15:56:26--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-23 15:56:26 (29.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [15]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [16]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [43]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [51]:
from openai import OpenAI
client  = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama'
)

In [52]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [53]:
rag("Write that this is a test")

'```markdown\n# Answer to General Course-related Questions: How do I use Git / GitHub for this course?\nTo utilize Git and GitHub during our analytics engineering with dbt module part of the Data Engineering Specialization offered by Google Developers Online Professional Certificate, please follow these steps after creating a basic understanding from "Git for Everybody" video tutorial. Here are detailed instructions incorporating best practices: Please note that I am providing general guidance as my capabilities do not include real-time interactions or live status displays such as those featured in the provided context.\n \nFirstly, create an account on GitHub if you haven\'t already done so by following their signup process outlined at www.github.com/join. Once your account is established and verified:\n1. Open a terminal window (Linux or Mac) where Git will be utilized. If using Windows with Cygwin, MSYS, MinTTY, ConEMU Linux Terminal emulator as referenced in the provided context, p

In [54]:
print(_)

```markdown
# Answer to General Course-related Questions: How do I use Git / GitHub for this course?
To utilize Git and GitHub during our analytics engineering with dbt module part of the Data Engineering Specialization offered by Google Developers Online Professional Certificate, please follow these steps after creating a basic understanding from "Git for Everybody" video tutorial. Here are detailed instructions incorporating best practices: Please note that I am providing general guidance as my capabilities do not include real-time interactions or live status displays such as those featured in the provided context.
 
Firstly, create an account on GitHub if you haven't already done so by following their signup process outlined at www.github.com/join. Once your account is established and verified:
1. Open a terminal window (Linux or Mac) where Git will be utilized. If using Windows with Cygwin, MSYS, MinTTY, ConEMU Linux Terminal emulator as referenced in the provided context, please d